### cookbook

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=.7)

In [6]:
chat([SystemMessage(content = "You are a helpful assistant that helps a user figure out what activity to do"),
      HumanMessage(content="I am bored and don't know what to do, I like outdoor activities")
])

AIMessage(content="How about going for a hike? It's a great way to enjoy the outdoors, get some exercise, and explore nature. You can choose a nearby trail or park, pack some snacks, and enjoy the fresh air while taking in the beautiful scenery.")

Documents

In [7]:
from langchain.schema import Document

In [8]:
Document(page_content="This is my document", 
         metadata={'my_doc_id': 1234,
            'my_doc_title': 'My Document Title'})


Document(page_content='This is my document', metadata={'my_doc_id': 1234, 'my_doc_title': 'My Document Title'})

Models

In [9]:
#lanuage model 
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-ada-001')

In [11]:
print(llm("what is the meaning of life?"))



The meaning of life is a question that has been asked by everyone from Socrates to Jesus. There is no one correct answer to this question.


chat model

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=1)

In [13]:
chat(
    [
        SystemMessage(content="You are an unhelpful AI bot that makes a joke at whatever the user says"),
        HumanMessage(content="I would like to go to New York, how should I do this?")
    ]
)

AIMessage(content="Why don't you try teleporting? Just think of New York and hope for the best!")

embeddings

In [14]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [16]:
text = "Hi! It's time for the beach"
text_embedding = embeddings.embed_query(text)
print (f"Here's a sample: {text_embedding[:5]}...")
print (f"Your embedding is length {len(text_embedding)}")

Here's a sample: [-0.00019600906371495044, -0.0031846734422911354, -0.0007734206914647712, -0.019472001962491225, -0.01509231901785424]...
Your embedding is length 1536


function calling

In [28]:
chat = ChatOpenAI(model='gpt-3.5-turbo-0613', temperature=1)

output = chat(messages=
     [
         SystemMessage(content="You are an helpful AI bot"),
         HumanMessage(content="What’s the weather like in Boston right now?")
     ],
     functions=[{
         "name": "get_current_weather",
         "description": "Get the current weather in a given location",
         "parameters": {
             "type": "object",
             "properties": {
                 "location": {
                     "type": "string",
                     "description": "The city and state, e.g. San Francisco, CA"
                 },
                 "unit": {
                     "type": "string",
                     "enum": ["fahrenheit"]
                 }
             },
             "required": ["location", "unit"]
         }
     }
     ]
)
print(output.additional_kwargs)

{'function_call': {'arguments': '{\n  "location": "Boston, MA",\n  "unit": "fahrenheit"\n}', 'name': 'get_current_weather'}}


prompt

In [31]:
llm = OpenAI(model_name='text-davinci-003')
from langchain import PromptTemplate

template ='''
I really want to do {activity} today. How do I get started?
I live in {location}.
'''

prompt = PromptTemplate(input_variables = ['activity', 'location'],
                        template = template)

final_prompt = prompt.format(activity='surfing', location='Seattle')

print(final_prompt)
print('*'*40)
print(f'LLM: {llm(final_prompt)}')


I really want to do surfing today. How do I get started?
I live in Seattle.

****************************************
LLM: 
If you're interested in surfing near Seattle, your best bet is to head to the Pacific Northwest's most popular surfing destination: the Oregon Coast. There are a variety of surf schools in the area which offer lessons for all levels of experience, as well as plenty of beaches where you can rent boards and wetsuits. It's also a great idea to connect with a local surf community to learn about the best spots and conditions.


In [32]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003")

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

# Examples of locations that nouns are found
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

In [35]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),#Embedding 
    Chroma,#Vector store
    k = 2)


In [38]:
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the location an item is usually found in",
    suffix = "Input: {noun}\nOutput:",
    input_variables=["noun"])

In [42]:
my_noun = "student"

print(similar_prompt.format(noun=my_noun))

Give the location an item is usually found in

Example Input: driver
Example Output: car

Example Input: pilot
Example Output: plane

Input: student
Output:


In [43]:
llm(similar_prompt.format(noun=my_noun), )

' classroom'

In [44]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [45]:
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```


In [59]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="welcom to califonya!")

print(promptValue)


You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```

% USER INPUT:
welcom to califonya!

YOUR RESPONSE:



In [47]:
llm_output = llm(promptValue)
llm_output

'```json\n{\n\t"bad_string": "welcom to califonya!",\n\t"good_string": "Welcome to California!"\n}\n```'

In [48]:
output_parser.parse(llm_output)

{'bad_string': 'welcom to califonya!', 'good_string': 'Welcome to California!'}

### Pydantic 

In [60]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Optional

class Person(BaseModel):
    """Identifying information about a person."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")

In [53]:
wes = Person(name="Wes", age=30, taco = "taco")

In [62]:
from langchain.chains.openai_functions import create_structured_output_chain


llm = ChatOpenAI(model='gpt-4-0613')

chain = create_structured_output_chain(Person, llm, prompt)
chain.run(
    "Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally."
)

Person(name='Sally', age=13, fav_food='spinach')

In [64]:
from typing import Sequence

class People(BaseModel):
    """Identifying information about all people in a text."""

    people: Sequence[Person] = Field(..., description="The people in the text")

In [65]:
chain = create_structured_output_chain(People, llm, prompt)
chain.run(
    "Sally is 13, Joey just turned 12 and loves spinach. Caroline is 10 years older than Sally."
)

People(people=[Person(name='Sally', age=13, fav_food='unknown'), Person(name='Joey', age=12, fav_food='spinach'), Person(name='Caroline', age=23, fav_food='unknown')])

In [66]:
import enum 

class Product(str, enum.Enum):
    """A product that can be purchased."""

    COFFEE = "coffee"
    TEA = "tea"
    SMOOTHIE = "smoothie"

    


In [71]:
class Products(BaseModel):
    """Identifying products that were mentioned in a text"""

    products: Sequence[Product] = Field(..., description="The products mentioned in a text")

In [72]:
chain = create_structured_output_chain(Products, llm, prompt)
chain.run("The coffee was good, but the tea was better. I didn't like the smoothie. I hated the chocolate.")

Products(products=[<Product.COFFEE: 'coffee'>, <Product.TEA: 'tea'>, <Product.SMOOTHIE: 'smoothie'>])